# NMF

### 1 - Methode de misse à jour multiplicative

In [1]:
import numpy as np
from scipy.optimize import nnls

In [2]:
def nmf_multiplicative_update(V, rank, max_iter=1000, tol=1e-10):
    
    m, n = V.shape
    W = np.random.rand(m, rank)
    H = np.random.rand(rank, n)

    for iteration in range(max_iter):
        # Calcul des mises à jour multiplicatives
        H_update = np.dot(W.T, V) / (np.dot(W.T, np.dot(W, H)) + 1e-10)
        H *= H_update
        H = np.maximum(H, 0) 
        
        W_update = np.dot(V, H.T) / (np.dot(W, np.dot(H, H.T)) + 1e-10)
        W *= W_update
        W = np.maximum(W, 0)  

        # Calcul de l'erreur de reconstruction
        reconstruction = np.dot(W, H)
        error = np.linalg.norm(V - reconstruction, ord='fro')
        
        # Arrêt si la tolérance est atteinte
        if error < tol:
            print(f"Convergence atteinte à l'itération {iteration + 1} avec erreur {error:.4f}")
            break
    
        # Signaler si max_iter est atteint
        if iteration == max_iter-1:
            print(f"Convergence stoppée à l'itération {iteration + 1}")
            
    print("Erreur :")
    print(error)

    return W, H


In [92]:
V = np.random.randint(100, size=(10, 10))

rank = 2  # Rang de la factorisation
max_iter = 100000
tol = 250

W, H = nmf_multiplicative_update(V, rank, max_iter, tol)

print("Matrice V :")
print(V)
print("\nMatrice W :")
print(W)
print("\nMatrice H :")
print(H)
print("\nReconstruction approximative de V :")
print(np.dot(W, H))


Convergence atteinte à l'itération 1 avec erreur 244.1950
Erreur :
244.19504790333409
Matrice V :
[[89 13 94 78 98 51 74 70 29 98]
 [13 64 94 57 66 70 62 46 69 70]
 [37 53 78  3 19 90 29 70 66 22]
 [90 39 53 28  3 32  1 42 31 59]
 [12  7 77 44 90 25 63 73 35 39]
 [92  9 96  7 62 32 79 56 56 23]
 [24 78 52 92 33 51 68 90 10 49]
 [94 41 22 29 58 78 90 95 29 76]
 [73 61 37 15 40 24 67 40 58 30]
 [73 63 88 30 70  9 31 92 27 81]]

Matrice W :
[[0.60764215 0.88762945]
 [0.55662664 0.71509703]
 [0.2883764  0.72073844]
 [0.04500767 0.67718049]
 [0.66851982 0.29462073]
 [0.57976543 0.51694099]
 [0.88218164 0.15068032]
 [0.84518636 0.40553183]
 [0.5271139  0.35263844]
 [0.30636705 0.88784913]]

Matrice H :
[[61.68168575 35.51163139 33.95978276 58.15428296 48.76723015 19.49996297
  74.30637538 91.65764951 46.30332616 55.95321352]
 [27.85939307 38.28921501 91.66800238 17.75641744 52.53224541 70.47978417
  33.00985105 27.0298079  14.64371291 48.27480007]]

Reconstruction approximative de V :
[[ 62.

### 2 - Methode des moindres carrées alternatifs

In [8]:
def nmf_als(V, rank, max_iter=1000, tol=1e-4):
   
    m, n = V.shape
    W = np.random.rand(m, rank)
    H = np.random.rand(rank, n)

    for iteration in range(max_iter):
        # Résolution pour H en fixant W
        for j in range(n):
            H[:, j], _ = nnls(W, V[:, j])
        H = np.maximum(H, 0)  
        
        # Résolution pour W en fixant H
        for i in range(m):
            W[i, :], _ = nnls(H.T, V[i, :])
        W = np.maximum(W, 0)  


        # Calcul de l'erreur de reconstruction
        reconstruction = np.dot(W, H)
        error = np.linalg.norm(V - reconstruction, ord='fro')


        # Critère d'arrêt basé sur la tolérance
        if error < tol:
            print(f"Convergence atteinte à l'itération {iteration + 1} avec erreur {error:.4f}")
            break

        # Signaler si max_iter est atteint
        if iteration == max_iter-1:
            print(f"Convergence stoppée à l'itération {iteration + 1}")
            
    print("Erreur :")    
    print(error)

    return W, H


In [76]:
V = np.random.randint(100, size=(10, 10))
rank = 2  # Rang de la factorisation
max_iter = 10000
tol = 250

W, H = nmf_als(V, rank, max_iter, tol)

print("Matrice V :")
print(V)
print("\nMatrice W :")
print(W)
print("\nMatrice H :")
print(H)
print("\nReconstruction approximative de V :")
print(np.dot(W, H))

Convergence atteinte à l'itération 1 avec erreur 208.2258
Erreur :
208.22578269034636
Matrice V :
[[21 77  2 83 42 46 22 91 26 34]
 [78 76 21 35 61 64 69 91 73  5]
 [42 45 40 85  4 73 19 44 91  6]
 [12 20 52 46 77 45 85 14  9 96]
 [40 96 45 57 88 94 95 29 98 67]
 [19 27 29 43 64  3 22 29 21 32]
 [47 90 96 97 55 63 61 68 45 74]
 [42  0 20 37 99 26 73 23 16 10]
 [28 33 76 41 79 59 12 35 23 15]
 [21 25 53 74 31 56 23 76 65  3]]

Matrice W :
[[0.         0.80413834]
 [0.12195759 0.95689469]
 [0.         0.8127346 ]
 [1.76157836 0.46752161]
 [0.90747803 1.05943131]
 [0.70089377 0.38845517]
 [0.18611827 1.14774023]
 [1.83577372 0.30835946]
 [0.68664259 0.59477276]
 [0.         0.76943148]]

Matrice H :
[[0.00000000e+00 0.00000000e+00 1.53042977e+01 4.74888283e-01
  3.82927305e+01 1.55656970e+00 3.15738088e+01 6.13872317e-03
  0.00000000e+00 5.93378215e+00]
 [4.56695605e+01 6.50109385e+01 5.08528486e+01 7.90611588e+01
  6.00051337e+01 6.79766650e+01 4.43990443e+01 6.40062737e+01
  6.20443713e

Le temps de calcul est clairement plus rapide mais pas d'amélioration de l'erreur. Cependant pour le seuil fixé à 250, il suffit d'une seule itération contrairement aux deux autres. le plus efficace mais pas le plus rapide.

### 3 - Descente de gradient

In [11]:
def nmf_gradient_descent(V, rank, max_iter=1000, tol=1e-4, learning_rate=0.001):
    
    m, n = V.shape
    W = np.random.rand(m, rank)
    H = np.random.rand(rank, n)

    for iteration in range(max_iter):
        # Calcul de l'erreur de reconstruction
        reconstruction = np.dot(W, H)
        error = V - reconstruction

        # Calcul des gradients
        grad_W = -2 * np.dot(error, H.T)
        grad_H = -2 * np.dot(W.T, error)

        # Mise à jour des matrices W et H
        W -= learning_rate * grad_W
        H -= learning_rate * grad_H

        W = np.maximum(W, 0)
        H = np.maximum(H, 0)

        # Calcul de l'erreur (norme de Frobenius)
        frobenius_error = np.linalg.norm(V - np.dot(W, H), ord='fro')


        # Critère d'arrêt basé sur la tolérance
        if frobenius_error < tol:
            print(f"Convergence atteinte à l'itération {iteration + 1} avec erreur {frobenius_error:.4f}")
            break
        
        # Signaler si max_iter est atteint
        if iteration == max_iter-1:
            print(f"Convergence stoppée à l'itération {iteration + 1}")
            
    print("Erreur :")
    print(frobenius_error)

    return W, H

In [62]:
V = np.random.randint(100, size=(10, 10))
rank = 2 # Rang de la factorisation
max_iter = 10000
tol = 250
learning_rate = 0.001

W, H = nmf_gradient_descent(V, rank, max_iter, tol, learning_rate)

print("Matrice V :")
print(V)
print("\nMatrice W :")
print(W)
print("\nMatrice H :")
print(H)
print("\nReconstruction approximative de V :")
print(np.dot(W, H))

Convergence stoppée à l'itération 10000
Erreur :
253.2229975852211
Matrice V :
[[67 65 75 17 56 61  9  8 10 15]
 [10 55  4 76  4 56 74 91 64 24]
 [43 97 78 62 38 94 23 77  4 42]
 [97 72 93 60 89 15 11 49 85 14]
 [60 77 40 37 84 25 95 67 58 22]
 [69 14 31 86 94 14 87 92 57 72]
 [ 4 63 46 15 10 93 23 84 83 75]
 [44 71  8 15 60 63 14 76 33 85]
 [55 28 95 38 45 44 36 43 85 89]
 [70 69 96 53 38 82 38 69 55 41]]

Matrice W :
[[ 1.41166199  6.94157694]
 [ 8.68962404  1.01339604]
 [ 7.00954131  5.18455099]
 [ 1.9941959  10.81159465]
 [ 5.12790021  6.85107512]
 [ 6.21861807  6.57057778]
 [ 9.47658465  1.32088228]
 [ 7.06008025  3.11245237]
 [ 5.24149736  6.57215546]
 [ 6.25272607  6.96945227]]

Matrice H :
[[ 1.25259399  6.78716078  2.29951178  5.0500886   1.74239424  9.14629566
   5.89740205 10.90711608  6.37491674  7.90085439]
 [10.18853976  6.06344856 10.12999694  5.00293515  9.60797484  2.12602335
   2.95652745  3.10441571  5.1761315   2.23487828]]

Reconstruction approximative de V :
[[ 72

On remarque que cet algorithme est beaucoup plus rapide en temps d'exécution que les autres avec learning_rate = 0.001 mais elle nécessite plus d'itérations pour atteindre la bonne matrice. D'ailleurs si on augmente le learning_rate, on obtient à un moment donné des matrices nulles pour W et H. Cependant toujours pas d'améliorations de l'estimation, sûrement que la meilleure estimation possible est atteinte.